In [ ]:
import pandas as pd
import numpy as np
from preprocess_data import preprocess_credit

from SDP import fairDimReductionFractional
from standard_PCA import std_PCA
from utils import getObj, get_recon_error, get_trace, get_optimal_error


In [ ]:
# standard PCA


## the data dumped from their matlab code (after they proprocessed their data)
dfCen = pd.read_csv("data/credit/data.csv")
df1 = pd.read_csv("data/credit/A.csv")
df2 = pd.read_csv("data/credit/B.csv")
print(len(df1), len(df2))  # make sure the numbers match the original paper's (~5300 high education)

B_whole = dfCen.values.T @ dfCen.values / len(dfCen)
n=len(B_whole[0])
P_all = std_PCA(B_whole,n)
X_van=[]
project_m = []
for d in range(0,n+1):
    X_van.append(P_all[:,:d] @ P_all[:,:d].T) 
    project_m.append(P_all[:,:d]) 

U_standard = []
for i in range(0, n+1):
    U_standard.append(project_m[i])
    
data1 = df1.values
data2 = df2.values
data = [data1, data2]

k=2
van_stats = pd.DataFrame();
for d in range(1, n+1):
    obj_dim_d={}
    obj_dim_d.update({'d':d})
    reconstruction_error_g = get_recon_error(n, k, d, data, U_standard[d])

    opt = get_optimal_error(n, k, d, data)
    for i in range(k):
        obj_dim_d.update({'grad_recons_err'+str(i):reconstruction_error_g[i]})
        obj_dim_d.update({'loss'+str(i):reconstruction_error_g[i]-opt[i]})  # already divided by len(data)

    van_stats = van_stats.append(pd.DataFrame(obj_dim_d,index=[d])) #add this info

#van_stats.to_csv('result_credit/standard_pca.csv',index=False) #if you want to save the result
van_stats #if you want to see what it is like

In [ ]:
# tilted fair PCA

import numpy as np

def get_loss(data, d, U, optimal_error):
    
    a = U @ U.T
    error = np.linalg.norm(data-np.dot(data, a), 'fro')**2 
    
    return (error-optimal_error)/len(data)


def my_fair_pca(data, d, init_u, initial_t):
    """
    data: the full data
    d: the target dimension
    """
    
    np.random.seed(1)
    full_dimension = len(data[0][0])
    U = init_u
    num_samples = len(data)
    lr = 0.001
  
    group1 = data[0]
    group2 = data[1]
    
    data1 = np.dot(group1.T, group1)
    data2 = np.dot(group2.T, group2) 
    
    P_all_1 = std_PCA(data1/len(data1), len(group1[0])) 
    p_1 = P_all_1[:,:d] @ P_all_1[:,:d].T
    optimal_error_1 = np.linalg.norm(group1-np.dot(group1, p_1), 'fro')**2
    
    
    P_all_2 = std_PCA(data2/len(data2), len(group2[0])) 
    p_2 = P_all_2[:,:d] @ P_all_2[:,:d].T
    optimal_error_2 = np.linalg.norm(group2-np.dot(group2, p_2), 'fro')**2
    
    
    for i in range(40000):

        q, r = np.linalg.qr(U)
        inverse_r =  np.linalg.inv(r)

        loss_1 = get_loss(group1, d, U, optimal_error_1)
        loss_2 = get_loss(group2, d, U, optimal_error_2)
            
        max_l = max(loss_1, loss_2)
        
        grad1 = -1*np.dot(data1, U)
        grad2 = -1*np.dot(data2, U)
        
        a = np.exp(t * (loss_1-max_l))
        b = np.exp(t * (loss_2-max_l))
        
        if i % 1000 == 0:
            print("losses:", loss_1, loss_2, "weights:", a/(a+b), b/(a+b))

        grad = a * grad1 + b * grad2   # sum of grads from all training samples
        grad = grad / (a*len(group1) + b*len(group2))

        # projected gradient descent
        U = np.dot(U - lr * grad, inverse_r) 
        
    return U
        

data1 = df1.values
data2 = df2.values
data = [data1, data2]

gradient_project_m = []

t = 10
for i in range(0, n+1):
    gradient_project_m.append(my_fair_pca(data, i, U_standard[i], t))

k=2 # number of groups
van_stats = pd.DataFrame();
for d in range(1, n+1):
    print(d)
    obj_dim_d = {}
    obj_dim_d.update({'d':d})
    reconstruction_error_g = get_recon_error(n, k, d, data, gradient_project_m[d])
    opt = get_optimal_error(n, k, d, data)
    
    for i in range(k):
        obj_dim_d.update({'grad_recons_err'+str(i):reconstruction_error_g[i]})
        obj_dim_d.update({'loss'+str(i):reconstruction_error_g[i]-opt[i]})

    van_stats = van_stats.append(pd.DataFrame(obj_dim_d,index=[d])) #add this info

#van_stats.to_csv('result_credit/fair_pca.csv',index=False) #if you want to save the result
van_stats # if you want to see what it is like